In [56]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer


In [57]:
train = pd.read_csv('./Data/our_train.csv')
X_train = train.drop('NObeyesdad', axis=1)
y_train = train['NObeyesdad']
labels = ['Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_I',
       'Obesity_Type_II', 'Obesity_Type_III', 'Overweight_Level_I',
       'Overweight_Level_II']

In [58]:
mati = {'id': 21372137, 'Gender' : 1, 'Age' : 20, 'Height' : 1.86, 'Weight' : 72, 'family_history_with_overweight' : 0, 'FAVC' : 0, 'FCVC' : 1,
       'NCP' : 3, 'CAEC' : 0.33, 'SMOKE' : 0, 'CH2O' : 0.7, 'SCC':0, 'FAF': 1, 'TUE':2, 'CALC':0.5, 'MTRANS':3}
gaspi = {'id': 21372138, 'Gender' : 1, 'Age' : 21, 'Height' : 1.74, 'Weight' : 64, 'family_history_with_overweight' : 0, 'FAVC' : 0, 'FCVC' : 1,
       'NCP' : 3, 'CAEC' : 0.33, 'SMOKE' : 0, 'CH2O' : 1.7, 'SCC':0, 'FAF': 1, 'TUE':2, 'CALC':0.5, 'MTRANS':3}

In [59]:
def calculate_bmi(data):
    data['BMI'] = data['Weight'] / (data['Height'] ** 2)
    return data.drop(['Weight', 'Height', 'id'], axis=1)

In [60]:
pipeline = Pipeline([
    ('calculate_bmi', FunctionTransformer(calculate_bmi)),
])

In [61]:
X_train = pipeline.fit_transform(X_train)
X_us = pd.merge(pd.DataFrame(mati, index=[0]), pd.DataFrame(gaspi, index=[1]), how='outer')
X_us = pipeline.transform(X_us)

In [62]:
X_us

,Gender,Age,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,BMI
0,1,20,0,0,1,3,0.33,0,0.7,0,1,2,0.5,3,20.811655
1,1,21,0,0,1,3,0.33,0,1.7,0,1,2,0.5,3,21.138856


In [63]:
cat_cols2 = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'MTRANS']
#change categorical columns to numerical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cat_cols2:
    X_train[col] = le.fit_transform(X_train[col])
    

y_train = le.fit_transform(y_train)

CAEC_dict = {'no': 0, 'Sometimes': 0.33, 'Frequently': 0.66, 'Always': 1 }
CALC_dict = {'no': 0, 'Sometimes': 0.5, 'Frequently': 1}

X_train['CAEC'] = X_train['CAEC'].map(CAEC_dict)
X_train['CALC'] = X_train['CALC'].map(CALC_dict)


In [64]:
X_train.head(3)

,Gender,Age,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,BMI
0,1,32.000000,1,0,2.000000,3.000000,0.33,0,1.000000,0,0.000000,2.000000,0.5,0,39.183673
1,0,19.783234,1,1,3.000000,3.000000,0.33,0,2.835622,0,1.655488,0.816986,0.5,3,40.325664
2,1,21.793724,1,1,2.921225,2.983201,0.33,0,2.184843,0,2.164472,0.000000,0.5,3,28.499889


In [65]:
cols_numerical = [ 'Gender', 'Age', 'family_history_with_overweight', 'FAVC', 'FCVC',
       'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS',
       'BMI']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_us = scaler.transform(X_us)
X_train = pd.DataFrame(X_train, columns=cols_numerical)
X_us = pd.DataFrame(X_us, columns=cols_numerical)
y_train = pd.DataFrame(y_train, columns=['NObeyesdad'])


In [66]:
labels = ['Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_I',
       'Obesity_Type_II', 'Obesity_Type_III', 'Overweight_Level_I',
       'Overweight_Level_II']

In [67]:
final_model = XGBClassifier(random_state = 1, colsample_bytree = 0.5, learning_rate = 0.01, max_depth = 7, n_estimators = 1000, 
                            subsample = 0.75, gamma = 0.5)
final_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.5, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [68]:
pred = final_model.predict(X_us)
print('mati: ' + labels[pred.tolist()[0]])
print('gaspi: ' + labels[pred.tolist()[1]])


mati: Normal_Weight
gaspi: Normal_Weight
